In [1]:
#Importing
import numpy as np
import random
import time
import warnings

# Suppress runtime warnings that can occur in benchmark functions
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [2]:
#######BENCHMARKING##############
def F1(x): return np.sum(x**2)
def F2(x): return np.sum(np.abs(x)) + np.prod(np.abs(x))
def F3(x):
    s = 0
    for i in range(len(x)): s += (np.sum(x[0:i+1]))**2
    return s
def F4(x): return np.max(np.abs(x))
def F5(x):
    dim = len(x)
    return np.sum(100 * (x[1:dim] - (x[0:dim-1]**2))**2 + (x[0:dim-1] - 1)**2)
def F6(x): return np.sum((np.floor(x + 0.5))**2)
def F7(x): return np.sum(np.arange(1, len(x)+1) * (x**4)) + np.random.uniform(0, 1)
def F8(x): return np.sum(-x * np.sin(np.sqrt(np.abs(x))))
def F9(x): return np.sum(x**2 - 10 * np.cos(2 * np.pi * x) + 10)
def F10(x):
    n = len(x)
    return -20*np.exp(-0.2*np.sqrt(np.sum(x**2)/n)) - np.exp(np.sum(np.cos(2*np.pi*x))/n) + 20 + np.e
def F11(x):
    return (1/4000) * np.sum(x**2) - np.prod(np.cos(x / np.sqrt(np.arange(1, len(x)+1)))) + 1
def u_penalty_function(val, a, k=100, m=4):
    """
    The penalty function u() as defined in the paper for F12 and F13.
    This is a direct mathematical translation without custom precision handling.
    """
    if val > a:
        return k * ((val - a) ** m)
    elif val < -a:
        return k * ((-val - a) ** m)
    else:
        return 0.0

def F12(x):
    """F12 implementation using the original u() penalty function."""
    n = len(x)
    y = 1 + (x + 1) / 4
    term1 = 10 * np.sin(np.pi * y[0])**2
    term2 = np.sum((y[:-1] - 1)**2 * (1 + 10 * np.sin(np.pi * y[1:])**2))
    term3 = (y[-1] - 1)**2
    penalty = np.sum([u_penalty_function(val, 10) for val in x])
    return (np.pi / n) * (term1 + term2 + term3) + penalty

def F13(x):
    """F13 implementation using the original u() penalty function."""
    n = len(x)
    term1 = 0.1 * np.sin(3 * np.pi * x[0])**2
    term2 = np.sum((x[:-1] - 1)**2 * (1 + np.sin(3 * np.pi * x[1:])**2))
    term3 = (x[-1] - 1)**2 * (1 + np.sin(2 * np.pi * x[-1])**2)
    penalty = np.sum([u_penalty_function(val, 5) for val in x])
    return term1 + 0.1 * (term2 + term3) + penalty
def F14(x):
    a = np.array([[-32, -16, 0, 16, 32]*5, [-32]*5 + [-16]*5 + [0]*5 + [16]*5 + [32]*5])
    s = sum(1 / (j + 1 + (x[0] - a[0, j])**6 + (x[1] - a[1, j])**6) for j in range(25))
    return (1/500 + s)**-1
def F15(x):
    a = np.array([0.00030,0.00138,0.00138,0.00138,0.00138,0.00030,0.00138,0.00138,0.00138,0.00138,0.00030])
    b = np.array([-2.477,-2.477,-2.477,-2.477,-2.477,2.477,2.477,2.477,2.477,2.477,2.477])
    return np.sum((a - x[0]*(b**2 + b*x[1]) / (b**2 + b*x[2] + x[3]))**2)
def F16(x): return 4*x[0]**2 - 2.1*x[0]**4 + (1/3)*x[0]**6 + x[0]*x[1] - 4*x[1]**2 + 4*x[1]**4
def F17(x): return (x[1]-(5.1/(4*np.pi**2))*x[0]**2+(5/np.pi)*x[0]-6)**2+10*(1-(1/(8*np.pi)))*np.cos(x[0])+10
def F18(x):
    return (1+(x[0]+x[1]+1)**2*(19-14*x[0]+3*x[0]**2-14*x[1]+6*x[0]*x[1]+3*x[1]**2)) * \
           (30+(2*x[0]-3*x[1])**2*(18-32*x[0]+12*x[0]**2+48*x[1]-36*x[0]*x[1]+27*x[1]**2))
def F19(x):
    a = np.array([[3, 10, 30], [0.1, 10, 35], [3, 10, 30], [0.1, 10, 35]])
    p = np.array([[0.3689, 0.1170, 0.2673], [0.4699, 0.4387, 0.7470], [0.1091, 0.8732, 0.5547], [0.03815, 0.5743, 0.8828]])
    c = np.array([1, 1.2, 3, 3.2])
    return -np.sum(c * np.exp(-np.sum(a * (x - p)**2, axis=1)))
def F20(x):
    a = np.array([[10,3,17,3.5,1.7,8], [0.05,10,17,0.1,8,14], [3,3.5,1.7,10,17,8], [17,8,0.05,10,0.1,14]])
    p = np.array([[0.1312,0.1696,0.5569,0.0124,0.8283,0.5886],[0.2329,0.4135,0.8307,0.3736,0.1004,0.9991],
                  [0.2348,0.1415,0.3522,0.2883,0.3047,0.6650],[0.4047,0.8828,0.8732,0.5743,0.1091,0.0381]])
    c = np.array([1, 1.2, 3, 3.2])
    return -np.sum(c * np.exp(-np.sum(a * (x - p)**2, axis=1)))
def _shekel(x, a, c, m): return sum(-1 / (np.sum((x - a[i])**2) + c[i]) for i in range(m))
def F21(x):
    a=np.array([[4,4,4,4],[1,1,1,1],[8,8,8,8],[6,6,6,6],[3,7,3,7]]); c=np.array([0.1,0.2,0.2,0.4,0.4])
    return _shekel(x,a,c,5)
def F22(x):
    a=np.array([[4,4,4,4],[1,1,1,1],[8,8,8,8],[6,6,6,6],[3,7,3,7],[2,9,2,9],[5,5,3,3]]); c=np.array([0.1,0.2,0.2,0.4,0.4,0.6,0.3])
    return _shekel(x,a,c,7)
def F23(x):
    a=np.array([[4,4,4,4],[1,1,1,1],[8,8,8,8],[6,6,6,6],[3,7,3,7],[2,9,2,9],[5,5,3,3],[8,1,8,1],[6,2,6,2],[7,3.6,7,3.6]]); c=np.array([0.1,0.2,0.2,0.4,0.4,0.6,0.3,0.7,0.5,0.5])
    return _shekel(x,a,c,10)

In [3]:
################ WaOA Implementation ######################
def WaOA(func, lb, ub, dim, N, T):
    # Initialization
    # Handle lb and ub if they are single values
    if not isinstance(lb, np.ndarray):
        lb = np.full(dim, lb)
    if not isinstance(ub, np.ndarray):
        ub = np.full(dim, ub)

    walrus_positions = np.random.uniform(0, 1, (N, dim)) * (ub - lb) + lb
    walrus_fitness = np.array([func(pos) for pos in walrus_positions])

    best_walrus_pos = walrus_positions[np.argmin(walrus_fitness)]
    best_walrus_fitness = np.min(walrus_fitness)

    # Main loop
    for t in range(1, T + 1):
        for i in range(N):
            current_pos = walrus_positions[i].copy()
            current_fitness = walrus_fitness[i]

            # Phase 1: Feeding strategy (Exploration)
            rand = np.random.rand(dim)
            I = np.random.randint(1, 3, dim)
            new_pos_p1 = current_pos + rand * (best_walrus_pos - I * current_pos) # Eq. (3)
            new_pos_p1 = np.clip(new_pos_p1, lb, ub)
            new_fitness_p1 = func(new_pos_p1)

            if new_fitness_p1 < current_fitness: # Eq. (4)
                current_pos = new_pos_p1
                current_fitness = new_fitness_p1

            # Phase 2: Migration
            k = np.random.choice(np.delete(np.arange(N), i))
            rand = np.random.rand(dim)
            I = np.random.randint(1, 3, dim)

            # Eq. (5)
            if walrus_fitness[k] < current_fitness:
                new_pos_p2 = current_pos + rand * (walrus_positions[k] - I * current_pos)
            else:
                new_pos_p2 = current_pos + rand * (current_pos - walrus_positions[k])

            new_pos_p2 = np.clip(new_pos_p2, lb, ub)
            new_fitness_p2 = func(new_pos_p2)

            if new_fitness_p2 < current_fitness: # Eq. (6)
                current_pos = new_pos_p2
                current_fitness = new_fitness_p2

            # Phase 3: Escaping and fighting predators (Exploitation)
            lb_local = lb / t # Eq. (8)
            ub_local = ub / t # Eq. (8)
            rand_val = np.random.rand()

            # Eq. (7)
            step = lb_local + (ub_local - rand_val * lb_local)
            new_pos_p3 = current_pos + step
            new_pos_p3 = np.clip(new_pos_p3, lb, ub)
            new_fitness_p3 = func(new_pos_p3)

            if new_fitness_p3 < current_fitness: # Eq. (9)
                current_pos = new_pos_p3
                current_fitness = new_fitness_p3

            walrus_positions[i] = current_pos
            walrus_fitness[i] = current_fitness

        # Update the global best walrus for the next iteration
        current_best_fitness = np.min(walrus_fitness)
        if current_best_fitness < best_walrus_fitness:
            best_walrus_fitness = current_best_fitness
            best_walrus_pos = walrus_positions[np.argmin(walrus_fitness)]

    return best_walrus_fitness

In [4]:
#Driver COde
if __name__ == '__main__':
    benchmarks = [
        {'name': 'F1', 'func': F1, 'dim': 30, 'lb': -100, 'ub': 100},
        {'name': 'F2', 'func': F2, 'dim': 30, 'lb': -10, 'ub': 10},
        {'name': 'F3', 'func': F3, 'dim': 30, 'lb': -100, 'ub': 100},
        {'name': 'F4', 'func': F4, 'dim': 30, 'lb': -100, 'ub': 100},
        {'name': 'F5', 'func': F5, 'dim': 30, 'lb': -30, 'ub': 30},
        {'name': 'F6', 'func': F6, 'dim': 30, 'lb': -100, 'ub': 100},
        {'name': 'F7', 'func': F7, 'dim': 30, 'lb': -1.28, 'ub': 1.28},
        {'name': 'F8', 'func': F8, 'dim': 30, 'lb': -500, 'ub': 500},
        {'name': 'F9', 'func': F9, 'dim': 30, 'lb': -5.12, 'ub': 5.12},
        {'name': 'F10', 'func': F10, 'dim': 30, 'lb': -32, 'ub': 32},
        {'name': 'F11', 'func': F11, 'dim': 30, 'lb': -600, 'ub': 600},
        {'name': 'F12', 'func': F12, 'dim': 30, 'lb': -50, 'ub': 50},
        {'name': 'F13', 'func': F13, 'dim': 30, 'lb': -50, 'ub': 50},
        {'name': 'F14', 'func': F14, 'dim': 2, 'lb': -65.536, 'ub': 65.536},
        {'name': 'F15', 'func': F15, 'dim': 4, 'lb': -5, 'ub': 5},
        {'name': 'F16', 'func': F16, 'dim': 2, 'lb': -5, 'ub': 5},
        {'name': 'F17', 'func': F17, 'dim': 2, 'lb': np.array([-5, 0]), 'ub': np.array([10, 15])},
        {'name': 'F18', 'func': F18, 'dim': 2, 'lb': -2, 'ub': 2},
        {'name': 'F19', 'func': F19, 'dim': 3, 'lb': 0, 'ub': 1},
        {'name': 'F20', 'func': F20, 'dim': 6, 'lb': 0, 'ub': 1},
        {'name': 'F21', 'func': F21, 'dim': 4, 'lb': 0, 'ub': 10},
        {'name': 'F22', 'func': F22, 'dim': 4, 'lb': 0, 'ub': 10},
        {'name': 'F23', 'func': F23, 'dim': 4, 'lb': 0, 'ub': 10},
    ]

    population_size = 30
    max_iterations = 500
    num_runs = 30

    print("="*60)
    print("Running Implementation of WaOA")
    print(f"Parameters: Pop Size={population_size}, Max Iterations={max_iterations}, Runs={num_runs}")
    print("="*60)

    for bench in benchmarks:
        name, func, dim, lb, ub = bench.values()
        print(f"\n--- Testing Function: {name} ---")

        results = []
        start_time = time.time()

        for run in range(num_runs):
            best_fitness = WaOA(
                func=func,
                dim=dim,
                N=population_size,
                T=max_iterations,
                lb=lb,
                ub=ub
            )
            results.append(best_fitness)
            print(f"  Run {run+1}/{num_runs}, Best Fitness: {best_fitness:.6e}", end='\r')

        end_time = time.time()
        avrg_fitness, std_fitness = np.mean(results), np.std(results)

        print("\n" + "-"*25)
        print(f"Results for {name}:")
        print(f"  AVRG : {avrg_fitness:.6e}")
        print(f"  STD :  {std_fitness:.6e}")
        print(f"  Time Taken:   {end_time - start_time:.2f} seconds")
        print("-"*25)

Running Implementation of WaOA
Parameters: Pop Size=30, Max Iterations=500, Runs=30

--- Testing Function: F1 ---
  Run 30/30, Best Fitness: 2.390179e-221
-------------------------
Results for F1:
  AVRG : 1.920704e-220
  STD :  0.000000e+00
  Time Taken:   54.79 seconds
-------------------------

--- Testing Function: F2 ---
  Run 30/30, Best Fitness: 8.147097e-115
-------------------------
Results for F2:
  AVRG : 1.085465e-113
  STD :  1.732220e-113
  Time Taken:   61.04 seconds
-------------------------

--- Testing Function: F3 ---
  Run 30/30, Best Fitness: 2.045824e-113
-------------------------
Results for F3:
  AVRG : 3.906826e-107
  STD :  2.103399e-106
  Time Taken:   234.20 seconds
-------------------------

--- Testing Function: F4 ---
  Run 30/30, Best Fitness: 5.722256e-102
-------------------------
Results for F4:
  AVRG : 1.177111e-99
  STD :  2.590491e-99
  Time Taken:   53.02 seconds
-------------------------

--- Testing Function: F5 ---
  Run 30/30, Best Fitness: 2